In [1]:
import torch
import nuqls.posterior as nqls
from scipy.stats import t
import torch.nn.functional as F
import numpy as np
import torch.utils.data as data
import torch.optim as optim
from scipy.stats import norm
from importlib import reload
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [31]:
def generate_data(data_dim, data_length, gt_label = False):
    x0 = torch.rand(data_length, data_dim) * 0.2
    x = torch.sin(x0)
    y = torch.sum(x, -1)
    if not gt_label:
        y += torch.normal(mean=0, std = 0.001, size = y.shape)
    return x0, y

class ImdbData(data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        return X, y

    def __len__(self):
        return len(self.y)

class Net(torch.nn.Module):
    def __init__(self, input_channel, hidden_layer):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_channel, hidden_layer)
        self.fc2 = torch.nn.Linear(hidden_layer, 1)
        self.hidden_layer = hidden_layer


    def forward(self, x ):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = x * np.sqrt(2/self.hidden_layer)
        x = self.fc2(x)
        return x

def weight_init(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.normal_(m.weight.data, mean=0.0, std=1.0)
        torch.nn.init.zeros_(m.bias)

def train(model, device, train_loader, optimizer, epoch, epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss_func = torch.nn.MSELoss()
        output = torch.reshape(output, target.shape)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()

    if epoch%500==0 or epoch == epochs:
        print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, loss.item()))
        
    return loss.item()
    

In [32]:
reload(nqls)
data_dim_array = [4]
data_length_array = [256]
lr = 5e-1
wd = 1e-10
batch_size = 128 
epochs = 1000
save_model = False

for data_dim, data_length in [(x, y) for x in data_dim_array for y in data_length_array]:
    print(f'\ndim: {data_dim}, n: {data_length}')
    cl = 0.05
    ci = 0.95
    ci2 = 0.9
    repeats = 50

    T_POINT = torch.ones(1,data_dim) * 0.1
    T_y = torch.sin(T_POINT)
    T_y = torch.sum(T_y, -1)
    T_POINT = T_POINT

    N_boot = 4
    # q1 = t.ppf(q = cl/2, df = 4, loc=0, scale=1)
    q1 = norm.ppf((1+ci)/2)
    # q2 = t.ppf(q = cl, df = 4, loc=0, scale=1)
    q2 = norm.ppf((1+ci2)/2)
    c_boot1, c_boot2 = 0, 0
    width_boot1, width_boot2 = 0, 0
    pred_boot = 0
    std_bootstrap_list = []
    mean_bootstrap_list = []

    for repeat in range(repeats):
        # generate the dataset
        # split the dataset into two part, A and B.
        X_train, y_train = generate_data(data_dim = data_dim, data_length = data_length)

        # generate dataset A
        A_start_index, A_end_index = 0, data_length
        A_set_index = list(range(A_start_index, A_end_index))
        X_Aset, Y_Aset = X_train[A_set_index], y_train[A_set_index]

        # get the initial network
        model0 = Net(input_channel = X_train.shape[-1], hidden_layer=data_length)
        model0.apply(weight_init)
        optimizer = optim.SGD(model0.parameters(), lr=lr, weight_decay=wd)
        PATH_ini = 'model0_initial.pth'
        torch.save(model0.state_dict(),PATH_ini)

        # train base models
        l = torch.nn.MSELoss()
        print('Training the base model')
        A_data_base = ImdbData(X_Aset, Y_Aset)
        A_loader_base = torch.utils.data.DataLoader(dataset=A_data_base,
                                                    batch_size=batch_size,
                                                    shuffle=True)
        for epoch in range(1, epochs + 1):
            tl = train(model0, device, A_loader_base, optimizer, epoch, epochs)
        print(f'train_error: {tl:.4}, test error: {l(model0(T_POINT),T_y).item():.4}')
        if save_model:
            PATH_final = 'model0_final.pth'
            torch.save(model0.state_dict(), PATH_final)

        # Create validation set
        X_val, y_val = generate_data(data_dim = data_dim, data_length = data_length)
        A_start_index, A_end_index = 0, data_length
        A_set_index = list(range(A_start_index, A_end_index))
        X_Aset, Y_Aset = X_val[A_set_index], y_val[A_set_index]
        val_set = ImdbData(X_Aset, Y_Aset)

        nuqls_posterior = nqls.Nuqls(model0, task='regression', full_dataset=True)
        # for epochs in torch.arange(50,400,50):
        #     res = nuqls_posterior.train(train=A_data_base, 
        #                         scale=1e-2, 
        #                         S=10, 
        #                         epochs=int(epochs.item()), 
        #                         lr=1, 
        #                         mu=0.9, 
        #                         verbose=True)
        #     print(f'test error: {l(nuqls_posterior.eval(T_POINT).mean(),T_y).item()}')
        # nuqls_posterior.HyperparameterTuning(validation=val_set, left=0.01, right=100, its=100, verbose=False)

        res = nuqls_posterior.train(train=A_data_base, 
                                scale=1e-2, 
                                S=10, 
                                epochs=200, 
                                lr=1, 
                                mu=0.9, 
                                verbose=True)
        
        nuqls_posterior.HyperparameterTuning(validation=val_set, left=0.01, right=100, its=200, verbose=False)
        
        pred = nuqls_posterior.eval(T_POINT)
        # mean_boot, std_boot = pred.mean(), pred.std()
        # width_boot1_scale = q1 * std_boot *2
        # scale = desired_width / width_boot1_scale

        scaled_pred = pred * (nuqls_posterior.scale_cal)
        mean_boot, std_boot = pred.mean(), scaled_pred.std()
        std_bootstrap_list.append(std_boot)
        mean_bootstrap_list.append(mean_boot)

        width_boot1 += q1 * std_boot *2
        width_boot2 += q2 * std_boot *2
        pred_boot += mean_boot
        if T_y >= mean_boot - std_boot * q1 and T_y <= mean_boot + std_boot * q1:
            c_boot1 += 1
        if T_y >= mean_boot - std_boot * q2 and T_y <= mean_boot + std_boot * q2:
            c_boot2 += 1

        # print(f'T_y: {T_y.item():.3}, mean: {mean_boot.item():.3}, MAP mean: {model0(T_POINT).item():.3}, ci: ({(mean_boot - std_boot * q1):.3},{(mean_boot + std_boot * q1):.3})')
        print(f'T_y: {T_y.item():.3}, nuql mean: {mean_boot.item():.3}, model mean: {model0(T_POINT).item():.3}, width: {(2* std_boot * q1):.3}')

        print('repeats {}, 95% c_values: {:.2f}, 90% c_values: {:.2f}, width: {:.4f}, pred: {:.4f}' 
            .format(repeat, c_boot1/len(std_bootstrap_list), c_boot2/len(std_bootstrap_list), width_boot2/len(std_bootstrap_list), pred_boot/len(std_bootstrap_list)))
        
    print('95% c_values: {}, width: {:.4f}, 90% c_values: {} width: {:.4f}, pred: {:.4f}' 
        .format(c_boot1/repeats, width_boot1/repeats, 
        c_boot2/repeats, width_boot2/repeats, pred_boot/repeats))




dim: 4, n: 256
Training the base model
Train Epoch: 500 	Loss: 0.000052
Train Epoch: 1000 	Loss: 0.000036
train_error: 3.644e-05, test error: 0.000276


100%|██████████| 200/200 [00:00<00:00, 540.99it/s, mean_loss=2.29e-5, resid_norm=1.1e-12, gpu_mem=0] 


Posterior samples computed!
Converged.
T_y: 0.399, nuql mean: 0.41, model mean: 0.416, width: 0.0337
repeats 0, 95% c_values: 1.00, 90% c_values: 1.00, width: 0.0283, pred: 0.4101
Training the base model
Train Epoch: 500 	Loss: 0.000106
Train Epoch: 1000 	Loss: 0.000066
train_error: 6.58e-05, test error: 0.0001324


100%|██████████| 200/200 [00:00<00:00, 538.98it/s, mean_loss=3.33e-5, resid_norm=1.98e-12, gpu_mem=0]


Posterior samples computed!
Converged.
T_y: 0.399, nuql mean: 0.411, model mean: 0.411, width: 0.0138
repeats 1, 95% c_values: 0.50, 90% c_values: 0.50, width: 0.0199, pred: 0.4107
Training the base model
Train Epoch: 500 	Loss: 0.000068
Train Epoch: 1000 	Loss: 0.000046
train_error: 4.593e-05, test error: 7.071e-05


...

Posterior samples computed!
Converged.
T_y: 0.399, nuql mean: 0.402, model mean: 0.404, width: 0.0149
repeats 47, 95% c_values: 0.98, 90% c_values: 0.92, width: 0.0182, pred: 0.4034
Training the base model
Train Epoch: 500 	Loss: 0.000044
Train Epoch: 1000 	Loss: 0.000043
train_error: 4.282e-05, test error: 2.199e-05


100%|██████████| 200/200 [00:00<00:00, 541.18it/s, mean_loss=2.17e-5, resid_norm=1.11e-12, gpu_mem=0]


Posterior samples computed!
Converged.
T_y: 0.399, nuql mean: 0.399, model mean: 0.404, width: 0.00895
repeats 48, 95% c_values: 0.98, 90% c_values: 0.92, width: 0.0180, pred: 0.4034
Training the base model
Train Epoch: 500 	Loss: 0.000101
Train Epoch: 1000 	Loss: 0.000049
train_error: 4.933e-05, test error: 4.32e-06


100%|██████████| 200/200 [00:00<00:00, 541.30it/s, mean_loss=2.55e-5, resid_norm=1.34e-12, gpu_mem=0]


Posterior samples computed!
Converged.
T_y: 0.399, nuql mean: 0.398, model mean: 0.397, width: 0.0171
repeats 49, 95% c_values: 0.98, 90% c_values: 0.92, width: 0.0179, pred: 0.4033
95% c_values: 0.98, width: 0.0214, 90% c_values: 0.92 width: 0.0179, pred: 0.4033


In [ ]:

dim: 8, n: 128
Training the base model
Train Epoch: 500 	Loss: 0.006802
Train Epoch: 1000 	Loss: 0.002597
Train Epoch: 1500 	Loss: 0.001516
Train Epoch: 2000 	Loss: 0.001043
Train Epoch: 2500 	Loss: 0.000766
Train Epoch: 3000 	Loss: 0.000585
Train Epoch: 3500 	Loss: 0.000459
Train Epoch: 4000 	Loss: 0.000368
Train Epoch: 4500 	Loss: 0.000302
Train Epoch: 5000 	Loss: 0.000251
Train Epoch: 5500 	Loss: 0.000212
Train Epoch: 6000 	Loss: 0.000182
Train Epoch: 6500 	Loss: 0.000157
Train Epoch: 7000 	Loss: 0.000138
Train Epoch: 7500 	Loss: 0.000121
Train Epoch: 8000 	Loss: 0.000107
Train Epoch: 8500 	Loss: 0.000096
Train Epoch: 9000 	Loss: 0.000086
Train Epoch: 9500 	Loss: 0.000077
Train Epoch: 10000 	Loss: 0.000070
Train Epoch: 10500 	Loss: 0.000063
Train Epoch: 11000 	Loss: 0.000057
Train Epoch: 11500 	Loss: 0.000053
Train Epoch: 12000 	Loss: 0.000048
Train Epoch: 12500 	Loss: 0.000044
Train Epoch: 13000 	Loss: 0.000041
Train Epoch: 13500 	Loss: 0.000038
Train Epoch: 14000 	Loss: 0.000035
Train Epoch: 14500 	Loss: 0.000033
Train Epoch: 15000 	Loss: 0.000030
Train Epoch: 15500 	Loss: 0.000028
Train Epoch: 16000 	Loss: 0.000026
Train Epoch: 16500 	Loss: 0.000024
Train Epoch: 17000 	Loss: 0.000023
Train Epoch: 17500 	Loss: 0.000021
Train Epoch: 18000 	Loss: 0.000020
Train Epoch: 18500 	Loss: 0.000019
Train Epoch: 19000 	Loss: 0.000017
Train Epoch: 19500 	Loss: 0.000016
Train Epoch: 20000 	Loss: 0.000015
train_error: 1.535e-05, test error: 0.002947
100%|██████████| 1000/1000 [00:13<00:00, 71.50it/s, mean_loss=7.47e-8, resid_norm=4.43e-16, gpu_mem=0]
Posterior samples computed!
Converged.
T_y: 0.799, nuql mean: 0.844, model mean: 0.853, width: 0.0796
repeats 0, 95% c_values: 0.00, 90% c_values: 0.00, width: 0.0668, pred: 0.8445
Training the base model
Train Epoch: 500 	Loss: 0.011700
Train Epoch: 1000 	Loss: 0.004441
Train Epoch: 1500 	Loss: 0.002316
Train Epoch: 2000 	Loss: 0.001494
Train Epoch: 2500 	Loss: 0.001090
Train Epoch: 3000 	Loss: 0.000843
Train Epoch: 3500 	Loss: 0.000675
Train Epoch: 4000 	Loss: 0.000549
Train Epoch: 4500 	Loss: 0.000453
Train Epoch: 5000 	Loss: 0.000376
Train Epoch: 5500 	Loss: 0.000316
Train Epoch: 6000 	Loss: 0.000267
Train Epoch: 6500 	Loss: 0.000228
Train Epoch: 7000 	Loss: 0.000196
Train Epoch: 7500 	Loss: 0.000170
Train Epoch: 8000 	Loss: 0.000148
Train Epoch: 8500 	Loss: 0.000129
Train Epoch: 9000 	Loss: 0.000114
Train Epoch: 9500 	Loss: 0.000100
Train Epoch: 10000 	Loss: 0.000089
Train Epoch: 10500 	Loss: 0.000079
Train Epoch: 11000 	Loss: 0.000071
Train Epoch: 11500 	Loss: 0.000063
Train Epoch: 12000 	Loss: 0.000057
Train Epoch: 12500 	Loss: 0.000052
Train Epoch: 13000 	Loss: 0.000047
Train Epoch: 13500 	Loss: 0.000042
Train Epoch: 14000 	Loss: 0.000038
Train Epoch: 14500 	Loss: 0.000035
Train Epoch: 15000 	Loss: 0.000032
Train Epoch: 15500 	Loss: 0.000029
Train Epoch: 16000 	Loss: 0.000027
Train Epoch: 16500 	Loss: 0.000025
Train Epoch: 17000 	Loss: 0.000023
Train Epoch: 17500 	Loss: 0.000021
Train Epoch: 18000 	Loss: 0.000019
Train Epoch: 18500 	Loss: 0.000018
Train Epoch: 19000 	Loss: 0.000017
Train Epoch: 19500 	Loss: 0.000015
Train Epoch: 20000 	Loss: 0.000014
train_error: 1.429e-05, test error: 0.00407
100%|██████████| 1000/1000 [00:14<00:00, 69.30it/s, mean_loss=4.5e-8, resid_norm=2.94e-16, gpu_mem=0]
Posterior samples computed!
Converged.
T_y: 0.799, nuql mean: 0.857, model mean: 0.862, width: 0.107
repeats 1, 95% c_values: 0.00, 90% c_values: 0.00, width: 0.0782, pred: 0.8506
Training the base model
Train Epoch: 500 	Loss: 0.010488
Train Epoch: 1000 	Loss: 0.004270
Train Epoch: 1500 	Loss: 0.002338
Train Epoch: 2000 	Loss: 0.001525
Train Epoch: 2500 	Loss: 0.001093
Train Epoch: 3000 	Loss: 0.000822
Train Epoch: 3500 	Loss: 0.000639
Train Epoch: 4000 	Loss: 0.000511
Train Epoch: 4500 	Loss: 0.000417
Train Epoch: 5000 	Loss: 0.000348
Train Epoch: 5500 	Loss: 0.000295
Train Epoch: 6000 	Loss: 0.000253
Train Epoch: 6500 	Loss: 0.000219
Train Epoch: 7000 	Loss: 0.000191
Train Epoch: 7500 	Loss: 0.000168
Train Epoch: 8000 	Loss: 0.000149
Train Epoch: 8500 	Loss: 0.000133
Train Epoch: 9000 	Loss: 0.000120
Train Epoch: 9500 	Loss: 0.000109
Train Epoch: 10000 	Loss: 0.000099

tensor([0.7987])

In [ ]:

(model0(T_POINT) - T_y)**2

tensor([[0.0004]], grad_fn=<PowBackward0>)

In [ ]:
cl = 0.05
repeats = 100

T_POINT = torch.ones(1,data_dim) * 0.1
T_y = torch.sin(T_POINT)
T_y = torch.sum(T_y, -1)
T_POINT = T_POINT

N_boot = 4
q1 = t.ppf(q = cl/2, df = N_boot, loc=0, scale=1)
q2 = t.ppf(q = cl, df = N_boot, loc=0, scale=1)
c_boot1, c_boot2 = 0, 0
width_boot1, width_boot2 = 0, 0
pred_boot = 0
std_bootstrap_list = []
mean_bootstrap_list = []

for repeat in range(repeats):
    pred = nuqls_posterior.eval(T_POINT)
    mean_boot, std_boot = pred.mean(), pred.std()
    std_bootstrap_list.append(std_boot)
    mean_bootstrap_list.append(mean_boot)
    width_boot1 += - q1 * std_boot *2
    width_boot2 += - q2 * std_boot *2
    pred_boot += mean_boot
    if T_y >= mean_boot + std_boot * q1 and T_y <= mean_boot - std_boot * q1:
        c_boot1 += 1
    if T_y >= mean_boot + std_boot * q2 and T_y <= mean_boot - std_boot * q2:
        c_boot2 += 1

    # print('repeats {}, 95% c_values: {:.2f}, 90% c_values: {:.2f}' 
    #     .format(repeat, c_boot1/len(std_bootstrap_list), c_boot2/len(std_bootstrap_list)))
    
print('95% c_values: {}, width: {:.4f}, 90% c_values: {} width: {:.4f}, pred: {:.4f}' 
    .format(c_boot1/repeats, width_boot1/repeats, 
    c_boot2/repeats, width_boot2/repeats, pred_boot/repeats))

95% c_values: 0.0, width: 0.0151, 90% c_values: 0.0 width: 0.0116, pred: 0.9224


T_POINT = torch.ones(1,args.data_dim) * 0.1
T_y = torch.sin(T_POINT)
T_y = torch.sum(T_y, -1)
T_POINT = T_POINT

N_boot = 4
q1 = t.ppf(q = args.cl/2, df = N_boot, loc=0, scale=1)
q2 = t.ppf(q = args.cl, df = N_boot, loc=0, scale=1)
c_boot1, c_boot2 = 0, 0
width_boot1, width_boot2 = 0, 0
pred_boot = 0
std_bootstrap_list = []
mean_bootstrap_list = []

for repeat in range(args.repeats):
    mean_boot, std_boot = bootstrap(args, T_POINT, T_y, N_boot, device)
    std_bootstrap_list.append(std_boot)
    mean_bootstrap_list.append(mean_boot)
    width_boot1 += - q1 * std_boot *2
    width_boot2 += - q2 * std_boot *2
    pred_boot += mean_boot
    if T_y >= mean_boot + std_boot * q1 and T_y <= mean_boot - std_boot * q1:
        c_boot1 += 1
    if T_y >= mean_boot + std_boot * q2 and T_y <= mean_boot - std_boot * q2:
        c_boot2 += 1

    print('repeats {}, 95% c_values: {:.2f}, 90% c_values: {:.2f}' 
        .format(repeat, c_boot1/len(std_bootstrap_list), c_boot2/len(std_bootstrap_list)))
    
print('95% c_values: {}, width: {:.4f}, 90% c_values: {} width: {:.4f}, pred: {:.4f}' 
    .format(c_boot1/args.repeats, width_boot1/args.repeats, 
    c_boot2/args.repeats, width_boot2/args.repeats, pred_boot/args.repeats))